In [472]:
import numpy as np
from scipy.interpolate import _bspl
import scipy.linalg as sl

def circle_vector(x,l=1,r=1):
    '''
    returns vector of nodes on circle
    '''
    #assert len(x) > max(l, r)
    dx = np.diff(x)
    t = np.zeros(len(x) + l + r)
    t[:l] = [x[0] - sum(dx[-i:]) for i in range(l,0,-1)]
    t[l:-r] = x
    t[-r:] = [x[-1] + sum(dx[:i]) for i in range(1,r+1)]
    return t

def B(x, k, i, t):
    if k == 0:
        if t[i] <= x < t[i+1]:
            return 1.0
        return 0.0
    if t[i+k] == t[i]:
        c1 = 0.0
    else:
        c1 = (x - t[i])/(t[i+k] - t[i]) * B(x, k-1, i, t)
    if t[i+k+1] == t[i+1]:
        c2 = 0.0
    else:
        c2 = (t[i+k+1] - x)/(t[i+k+1] - t[i+1]) * B(x, k-1, i+1, t)
    return c1 + c2

def bspline(x, t, c, k):
    n = len(t) - k - 1
    assert (n >= k+1) and (len(c) >= n)
    return sum(c[i] * B(x, k, i, t) for i in range(n))

def woodbury(A, ur, ll, b, k):
    '''
    Implementation of Woodbury algorithm applied to banded
    matrices with two blocks in upper right and lower left
    corners.
    
    Parameters
    ----------
    A : 2-D array, shape(k, n)
        Matrix of diagonals of original matrix(see 
        'solve_banded' documentation).
    ll : 2-D array, shape(bs,bs)
        Lower left block matrix.
    ur : 2-D array, shape(bs,bs)
        Upper right block matrix.
    b : 1-D array, shape(1,n)
        Vector of constant terms of the SLE.
        
    Returns
    -------
    c : 1-D array, shape(1,n)
        Solution of the original SLE.
        
    Notes
    -----
    SLE - system of linear equations.
    
    'n' should be greater than 'k', otherwise corner block
    elements will intersect diagonals.
    '''
    k_odd = (k+1) % 2
    bs = int((k-1)/2) + k_odd
    n = A.shape[1] + 1
    #U = np.zeros((n-1, k-1))
    #V = np.zeros((k-1, n-1)) # V transpose
    U = np.zeros((n-1, k-k%2))
    V = np.zeros((k-k%2, n-1)) # V transpose 

    # upper right

    U[:bs, :bs] = ur
    for j in range(bs): 
        V[j, -bs+j] = 1

    # lower left

    U[-bs:, -bs:] = ll
    for j in range(bs): 
        V[-bs+j, j] = 1
    
    Z = sl.solve_banded((bs-k_odd, bs), A, U[:, 0]) # z0
    Z = np.expand_dims(Z, axis=0)
    
    #for i in range(1, k-1):
    for i in range(1, k-k%2):
        zi = sl.solve_banded((bs-k_odd, bs), A, U[:, i])
        zi = np.expand_dims(zi, axis=0)
        Z = np.concatenate((Z, zi), axis=0)

    Z = Z.transpose()
    #H = sl.inv(np.identity(k - 1) + V @ Z)
    H = sl.inv(np.identity(k - k%2) + V @ Z)

    y = sl.solve_banded((bs-k_odd, bs), A, b)
    c = y - Z @ (H @ (V @ y))

    return np.concatenate((c[-bs + k_odd:], c, c[: bs + 1]))

def make_matrix(x,y,t,k):
    '''
    Returns diagonals and block elements of the matrix
    formed by points (x, y) for coefficients for B-Spline
    curve.
    
    Parameters
    ----------
    x : 1-D array, shape(n,)
        x coordinate of points.
    y : 1-D array, shape(n,).
        y coordiante of points.
    t : 1-D array, shape(n+2*k,)
        Node vector.
    k : int
        Degree of B-splines.
    Returns
    -------
    A : 2-D array, shape(k,n-1)
        Diagonals of the original matrix of SLE.
    ur : 2-D array, shape(offset,offset)
        Upper right block of the original matrix of SLE.
    ll : 2-D array, shape(offset,offset)
        Lower left block of the original matrix of SLE.
    Notes
    -----
    SLE - system of linear equations.
    
    'n' should be greater than 'k', otherwise corner block
    elements will intersect diagonals.
    offset = (k - 1) / 2
    '''
    n = x.size
    yc = np.copy(y)
    yc = yc[:-1]
    A = np.zeros((k, n - 1))
    for i in range(n-1):
        A[:,i] = _bspl.evaluate_all_bspl(t, k, x[i], i + k)[:-1][::-1]
    offset = int((k-1)/2) + (k + 1) % 2
    ur = np.zeros((offset,offset))
    ll = np.zeros((offset,offset))
    for i in range(1,offset + 1):
        A[offset - i] = np.roll(A[offset - i],i)
        if k % 2 == 1 or i < offset:
            A[offset + i] = np.roll(A[offset + i],-i)
            ur[-i:,i-1] = np.copy(A[offset + i,-i:])
        ll[-i,:i] = np.copy(A[offset - i,:i])
    ur = ur.T
    for i in range(1,offset):
        ll[:,i] = np.roll(ll[:,i],i)
        ur[:,-i-1] = np.roll(ur[:,-i-1],-i)
    return A, ur, ll

In [517]:
n = 13
k = 3
x = np.sort(np.random.random_sample(n) * 20 - 10)
y = np.random.random_sample(n) * 200 - 100
y[-1] = y[0]
t = circle_vector(x,k,k)

In [518]:
matr = np.zeros((n+k,n+k))
for i in range(n):
    matr[i + k - 1,i:i+k+1] = _bspl.evaluate_all_bspl(t, k, x[i], i + k)
for i in range(k-1):
    matr[i,:k + 1] = _bspl.evaluate_all_bspl(t, k, x[0], k, nu=i+1)
    matr[i, -k-1:] = -_bspl.evaluate_all_bspl(t, k, x[-1], n+k-1, nu=i+1)
matr = matr[:-1,:-1]
b = np.zeros_like(matr[:,0])
b[k-1:] = y

In [519]:
from scipy.sparse.linalg import splu
from scipy.sparse import csc_matrix

print(np.linalg.det(matr))

lu = splu(csc_matrix(matr))
diagL = lu.L.diagonal()
diagU = lu.U.diagonal()
'''swap_sign = minimumSwaps(lu.perm_r)
print(swap_sign)
sign = swap_sign*np.sign(diagL).prod()*np.sign(diagU).prod()
print(sign)'''
print(diagL.prod()*diagU.prod()) #*sign)

-1.6558019614194445e-06
1.655801961419443e-06


In [520]:
c = np.linalg.solve(matr,b)
A,u,l = make_matrix(x,y,t,k)
sol = woodbury(A,u,l,y[:-1],k)

In [521]:
with np.printoptions(precision=2,suppress=True):
  print(A, '\n', u,'\n', l)

[[0.07 0.   0.3  0.4  0.02 0.16 0.75 0.04 0.   0.44 0.02 0.82]
 [0.18 0.52 0.58 0.65 0.53 0.24 0.94 0.14 0.55 0.61 0.18 0.92]
 [0.18 0.01 0.33 0.32 0.01 0.02 0.85 0.01 0.37 0.01 0.01 0.82]] 
 [[0.82]] 
 [[0.07]]


In [522]:
with np.printoptions(precision=2,suppress=True):
  print(matr[k-1:])

[[0.82 0.18 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.  ]
 [0.   0.18 0.52 0.3  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.  ]
 [0.   0.   0.01 0.58 0.4  0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.  ]
 [0.   0.   0.   0.33 0.65 0.02 0.   0.   0.   0.   0.   0.   0.   0.
  0.  ]
 [0.   0.   0.   0.   0.32 0.53 0.16 0.   0.   0.   0.   0.   0.   0.
  0.  ]
 [0.   0.   0.   0.   0.   0.01 0.24 0.75 0.   0.   0.   0.   0.   0.
  0.  ]
 [0.   0.   0.   0.   0.   0.   0.02 0.94 0.04 0.   0.   0.   0.   0.
  0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.85 0.14 0.   0.   0.   0.   0.
  0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.01 0.55 0.44 0.   0.   0.
  0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.37 0.61 0.02 0.   0.
  0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.01 0.18 0.82 0.
  0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.01 0.92 0.07
  0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   

In [523]:
np.allclose(sol, c)

True

In [524]:
c, sol

(array([   80.26955918,   -82.51644297,   245.4003261 ,  -200.10177734,
          249.79200827,    24.59973891,  -460.6427198 ,   171.03844637,
        -1574.64938608,   112.2806238 ,   -13.10608161,   156.22558447,
           80.26955918,   -82.51644297,   245.4003261 ]),
 array([   80.26955918,   -82.51644297,   245.4003261 ,  -200.10177734,
          249.79200827,    24.59973891,  -460.6427198 ,   171.03844637,
        -1574.64938608,   112.2806238 ,   -13.10608161,   156.22558447,
           80.26955918,   -82.51644297,   245.4003261 ]))

In [525]:
np.allclose((matr @ c)[k-1:], y)

True

In [526]:
(matr @ c)

array([ 0.00000000e+00,  5.68434189e-14,  5.15696938e+01,  5.32141214e+01,
       -1.25539401e+01,  9.58740038e+01,  1.90474084e+01,  1.56680225e+01,
        8.17103119e+01, -8.07777313e+01,  3.32355437e+01,  3.68487146e+01,
        9.33113805e+01,  7.01575716e+01,  5.15696938e+01])

In [527]:
y

array([ 51.56969383,  53.21412136, -12.55394008,  95.87400379,
        19.04740837,  15.66802246,  81.7103119 , -80.77773135,
        33.2355437 ,  36.84871461,  93.31138051,  70.15757156,
        51.56969383])

In [528]:
np.allclose((matr @ sol)[k-1:] , y)

True

In [529]:
(matr @ sol)[k-1:]

array([ 51.56969383,  53.21412136, -12.55394008,  95.87400379,
        19.04740837,  15.66802246,  81.7103119 , -80.77773135,
        33.2355437 ,  36.84871461,  93.31138051,  70.15757156,
        51.56969383])

In [530]:
y

array([ 51.56969383,  53.21412136, -12.55394008,  95.87400379,
        19.04740837,  15.66802246,  81.7103119 , -80.77773135,
        33.2355437 ,  36.84871461,  93.31138051,  70.15757156,
        51.56969383])